In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
from subprocess import run
import re
from statistics import mean

In [5]:
# set up filenames and paths
file = "SealedCore"
nd_directory = "~/Documents/Enceladus/1D_files"
bm_directory =  "~/Documents/Enceladus/1D_files/output"

nd_filename = f"{nd_directory}/{file}.nd"
bm_filename = f"{bm_directory}/{file}.bm"

# header information to use in .bm file - change as needed
citation = "Velocity model for Enceladus according to Dapré & Irving (2025)"
anelastic = "false"
aniso = "false"
units = "m"
columns = "radius rho vp vs"

In [6]:
# read in the input named discontinuities (.nd) file
def Read_nd_mod(file_name):
    print(file_name)
    with open(file_name, 'r') as data:
        dep = []
        rawvp = []
        rawvs = []
        rawrho = []
        layernames = []
        bounds = []
        # add regolith to the layer names to keep AxiSEM meshers happy
        layernames.insert(0, "regolith")
        bounds.insert(0, 0)
        i = -1   # keeps track of which input file line we're on
        k = -1   # keeps track of the length of output arrays
        j = 1    # tracks how many layer names have been written
        for line in data:
            i = i+1
            line = line.replace('\n','')
            layernames.append(line)
            bounds.append(i+1)
            try :
                float(line[0]) # i.e. execute this block if the first character of the line is a number
                layernames.pop()
                bounds.pop()
                p = line.split()
                dep.append(float(p[0]))
                rawvp.append(float(p[1]))
                rawvs.append(float(p[2]))
                rawrho.append(float(p[3]))
                k = k+1
            except:
                pass
            
            if k > 0: # i.e. if dep contains at least 2 values
                # if depth is the same as previous entry (indicating discontinuity) and the layer has not been named
                if dep[k] == dep[k-1] and isname == 0:
                    layernames.append(f"unnamed-layer-{len(layernames)+1}")
                    bounds.append(i+1)
                    j = j+1
                    
            if layernames[-1] in line: # i.e. if this line is a layername, stop the unnamed layer block from triggering next run
                isname = 1
            else:
                isname = 0
                    
        bounds.append(i+j+1)    # identify last line
        r = max(dep) # obtain radius
        
            
    return dep, rawvp, rawvs, rawrho, bounds, r, layernames

In [7]:
rawdep, rawvp, rawvs, rawrho, bounds, rmax, layernames = Read_nd_mod(nd_filename)

print(rawdep)
print(layernames)
print(rmax)
print(bounds)

C:/Users/mercu/Downloads/1D_files/CadekEPSCBrantutW2.nd
[0.0, 0.41, 0.41, 1.63, 2.86, 4.08, 5.31, 6.53, 7.75, 8.98, 10.2, 11.43, 12.65, 13.88, 15.1, 16.33, 17.55, 18.78, 20.0, 20.0, 69.6, 69.6, 69.65, 78.55, 87.69, 96.82, 105.95, 115.09, 124.22, 133.36, 142.49, 151.62, 160.76, 169.89, 179.03, 188.16, 197.29, 206.43, 215.56, 224.7, 233.83, 242.97, 252.1]
['regolith', 'mantle', 'outer-core', 'inner-core']
252.1
[0, 3, 21, 24, 47]


In [8]:
# function to format header for the .bm file
def Make_bm_header(citation, anelastic, aniso, columns, units):
    lines = [f"#{citation}", f"ANELASTIC   {anelastic}", f"ANISOTROPIC {aniso}", f"UNITS      {units}", f"COLUMNS    {columns}\n"]
    
    return lines

In [9]:
header = Make_bm_header(citation, anelastic, aniso, columns, units)
print(header)

['#Velocity model for Enceladus according to Cadek16Enc', 'ANELASTIC   false', 'ANISOTROPIC false', 'UNITS      m', 'COLUMNS    radius rho vp vs\n']


In [10]:
# function to write .bm model
def Make_bm_mod(rawdep, rawvp, rawvs, rawrho, bounds, rmax, layernames, header, output_filename, units):
    j = 0
    k = 0
    n = 0
    lines = []
    # sort out scaling units
    if units =='m':
        m = 1
    elif units =='km':
        m = 0
    # begin constructing file
    for i in range(0,len(layernames)):
        if layernames[j] != "mantle": # avoiding writing the mantle layer name for AxiSEM mesher compatibility
            if "unnamed-layer" not in layernames[j]: # remove unnamed layer names from .bm file output
                lines.append(f"# {layernames[j]}")
        for n in range(bounds[j]-j,bounds[j+1]-j-1): # select all data until the next boundary
            if layernames[j] =="mantle" and rawdep[n-1]==rawdep[n]:
                del lines[-1] # remove duplicate line where mantle layer name has been skipped
            vp = round(rawvp[n]*1000**m,2)
            vs = round(rawvs[n]*1000**m,2)
            rho = round(rawrho[n]*1000**m,2)
            dep = round((rmax-rawdep[n])*1000**m,3) # convert depth to radius
            lines.append(f"   {dep}   {rho}   {vp}   {vs}")
        j = j+1 # increment the layer
    data = header + lines
    print(data)
     
    with open(output_filename, 'w') as fout:
        for item in data:
            fout.write("%s\n" % item)
    
    return data

In [11]:
output = Make_bm_mod(rawdep,rawvp,rawvs,rawrho,bounds,rmax,layernames,header,bm_filename, units)

['#Velocity model for Enceladus according to Cadek16Enc', 'ANELASTIC   false', 'ANISOTROPIC false', 'UNITS      m', 'COLUMNS    radius rho vp vs\n', '# regolith', '   252100.0   930.0   4250.4   4250.4', '   251690.0   930.0   4241.8   4241.8', '   250470.0   930.0   4215.9   4215.9', '   249240.0   930.0   4189.7   4189.7', '   248020.0   930.0   4163.4   4163.4', '   246790.0   930.0   4136.7   4136.7', '   245570.0   930.0   4109.7   4109.7', '   244350.0   930.0   4082.5   4082.5', '   243120.0   930.0   4055.0   4055.0', '   241900.0   930.0   4027.3   4027.3', '   240670.0   930.0   3999.2   3999.2', '   239450.0   930.0   3970.9   3970.9', '   238220.0   930.0   3942.2   3942.2', '   237000.0   930.0   3913.4   3913.4', '   235770.0   930.0   3884.1   3884.1', '   234550.0   930.0   3854.6   3854.6', '   233320.0   930.0   3824.8   3824.8', '   232100.0   930.0   3794.6   3794.6', '# outer-core', '   232100.0   1030.0   1400.0   1400.0', '   182500.0   1030.0   1400.0   1400.0',